# Yemeksepeti Data Collection

This notebook aims to employ various web scraping techniques to gather data on restaurants from the popular platform "Yemeksepeti". The collected data will be stored in the JSON format within the designated directory named `collected_data`.

The dataset will encompass a wide range of information pertaining to the restaurants, encompassing details such as their menu items, pricing, customer comments, ratings, and corresponding timestamps.

Subsequently, the primary goal is to conduct semantic analysis on the acquired dataset, with the aim of gaining deeper insights into consumer behavior patterns within the context of Turkey.

In [72]:
# import libraries

import requests
import json
import os


In [ ]:
# directories 

COLLECTED_DATA_DIR = '../collected_data/'

## A List of Resturants in every city in Turkey 

In this section, our goal is to gather a comprehensive list of all restaurants in every city in Turkey, complete with detailed information such as their address, budget, cuisine offerings, and most importantly, the URLs leading to their respective pages on Yemeksepeti.

To accomplish this task, we will send a specific request to Yemeksepeti's database and parse the response into a JSON format. This will enable us to efficiently organize and explore the gathered information, providing valuable insights

In [73]:
# get the 
def get_resturants_data_for_city(city_id):
    # Yemeksepeti uses the Delivery Hero servers to store it's data, hence we can get resturant's data by sending a request to delivery hero
    # the arguments ...&offset=0&limit=500&... can be added to the url to specify the data request size
    yemeksepeti_request_url = f"https://disco.deliveryhero.io/listing/api/v1/pandora/vendors?language_id=2&vertical=restaurants&country=tr&include=characteristics&configuration=Variant1&sort=&city_id={city_id}"
    
    # the headers are acquired from the requests from yemeksepeti's webpage
    # perseus-client-id and perseus-seesion-id might needed be updated occasionally
    headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'dnt': '1',
    'origin': 'https://www.yemeksepeti.com',
    'perseus-client-id': '1683402284518.770138949646337300.d4t6e4b8h2',
    'perseus-session-id': '1686229407319.641123708405837200.lsjp7m4zlk', #'perseus-session-id': '1686226160511.373585736230385800.r7egjnpz9u',
    'referer': 'https://www.yemeksepeti.com/',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'x-disco-client-id': 'web',
    'x-fp-api-key': 'volo '
    }

    # send the request and store the response
    response = requests.get(yemeksepeti_request_url, headers=headers)
    
    return response


In [74]:
# A list of all cities in turkey with their index - 1 corresponding to the city's id (il kodu)
# e.g. cities[33] == cities[34 - 1] == "Istanbul" 
cities = ["Adana", "Adıyaman", "Afyon", "Ağrı", "Amasya", "Ankara", "Antalya", "Artvin", "Aydın", "Balıkesir", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane", "Hakkari", "Hatay", "Isparta", "İçel (Mersin)", "İstanbul", "İzmir", "Kars", "Kastamonu", "Kayseri", "Kırklareli", "Kırşehir", "Kocaeli", "Konya", "Kütahya", "Malatya", "Manisa", "Kahramanmaraş", "Mardin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Tekirdağ", "Tokat", "Trabzon", "Tunceli", "Şanlıurfa", "Uşak", "Van", "Yozgat", "Zonguldak", "Aksaray", "Bayburt", "Karaman", "Kırıkkale", "Batman", "Şırnak", "Bartın", "Ardahan", "Iğdır", "Yalova", "Karabük", "Kilis", "Osmaniye", "Düzce"]

In [77]:
file_list = os.listdir(COLLECTED_DATA_DIR)

# Save the data of resturants in all cities as seperate json files
for city_id, city_name in enumerate(cities):
    filename = f'{city_id + 1}_yemeksepeti_{city_name}_resturants_data.json'

    # check for missing cities
    if (filename not in file_list):
        # Fetch the data for a specific city
        print(f'({city_id+1} / {len(cities)}) Fetching the data for {city_name} resturants:')
        resturants_data = get_resturants_data_for_city(city_id + 1)

        # save the response data as json
        print("saving the json data...")
        data_export_directory = COLLECTED_DATA_DIR + filename
        with open(data_export_directory, 'wb') as of:
            of.write(resturants_data.content)

        print(f"succefully exported the data. path: `{data_export_directory}`\n")

print("Finished.")

Finished.
